# insert_ads_campaign_prd_in_store_full

- Name: insert_ads_campaign_prd_in_store_full
- Description: campaign summary based on product_id, purchase type only offline
- Data modeling reference: https://docs.google.com/spreadsheets/d/1FlbiZEBue2SCAUo1WOkdtX5WbEDyDrZ9B2uiZUSiRio/edit?gid=2043216767#gid=2043216767
- Target Table: ads_campaign_prd_in_store_full
- Source Table: dws_campaign_cost_sales_prd_d_inc
- Created by: alvinxyzhang
- Created Date: 2025-05-02
- Version: v1.0

In [1]:
from google.cloud import bigquery
import datetime
import pandas as pd

## Scheduled Task & Partition Table

In [21]:
# Construct a BigQuery client object.
client = bigquery.Client()

table_schema = [
        bigquery.SchemaField("product_id", "INTEGER"),
        bigquery.SchemaField("product_name", "STRING"),
        bigquery.SchemaField("brand", "STRING"),
        bigquery.SchemaField("is_brand_halo_effect", "INTEGER"),
        bigquery.SchemaField("category", "STRING"),
        bigquery.SchemaField("is_category_halo_effect", "INTEGER"),
        bigquery.SchemaField("is_included_in_campaign", "INTEGER"),
        bigquery.SchemaField("campaign_id", "INTEGER"),
        bigquery.SchemaField("campaign_name", "STRING"),
        bigquery.SchemaField("campaign_type", "STRING"),
        bigquery.SchemaField("start_date", "DATE"),
        bigquery.SchemaField("end_date", "DATE"),
        bigquery.SchemaField("estimated_budget", "INTEGER"),
        bigquery.SchemaField("approved_budget", "INTEGER"),
        bigquery.SchemaField("real_spent_budget", "INTEGER"),
        bigquery.SchemaField("discount", "FLOAT"),
        bigquery.SchemaField("count_product", "INTEGER"),
        bigquery.SchemaField("cost_in_store", "FLOAT"),
        bigquery.SchemaField("cost_flyer", "FLOAT"),
        bigquery.SchemaField("cost_community", "FLOAT"),
        bigquery.SchemaField("cost_loyalty", "FLOAT"),
        bigquery.SchemaField("cost_push", "FLOAT"),
        bigquery.SchemaField("cost_ad_total", "FLOAT"),
        bigquery.SchemaField("baseline_revenue", "FLOAT"),
        bigquery.SchemaField("baseline_cost", "FLOAT"),
        bigquery.SchemaField("campaign_revenue", "FLOAT"),
        bigquery.SchemaField("campaign_cost", "FLOAT"),
        bigquery.SchemaField("estimated_revenue", "FLOAT"),
        bigquery.SchemaField("incremental_revenue", "FLOAT"),
        bigquery.SchemaField("halo_cost_in_store", "FLOAT"),
        bigquery.SchemaField("halo_cost_flyer", "FLOAT"),
        bigquery.SchemaField("halo_cost_community", "FLOAT"),
        bigquery.SchemaField("halo_cost_loyalty", "FLOAT"),
        bigquery.SchemaField("halo_cost_push", "FLOAT"),
        bigquery.SchemaField("halo_cost_ad_total", "FLOAT")
    ]

insert_query = f"""

    WITH post_campaign_tbl AS (
      SELECT product_id
            , SUM(number_of_items) number_of_items_sum
            , AVG(retail_price) retail_price
            , AVG(retail_cost) retail_cost
            , SUM(total_price) total_price_sum
            , SUM(total_cost) total_cost_sum
            , AVG(retail_discount_price) retail_discount_price
            , SUM(total_discount_price) total_discount_price_sum
      FROM `positive-karma-457703-i3.retail_dashboard.dws_campaign_cost_sale_prd_d_inc`
      WHERE `date` >= @post_campaign_start_date_ and `date` <= @post_campaign_end_date_
        AND purchase_type = 'instore'
      GROUP BY product_id
    )
    , baseline_tbl AS (
      SELECT product_id
          , SUM(number_of_items) number_of_items_sum
          , AVG(retail_price) retail_price
          , AVG(retail_cost) retail_cost
          , SUM(total_price) total_price_sum
          , SUM(total_cost) total_cost_sum
          , AVG(retail_discount_price) retail_discount_price
          , SUM(total_discount_price) total_discount_price_sum
      FROM `positive-karma-457703-i3.retail_dashboard.dws_campaign_cost_sale_prd_d_inc`
      WHERE `date` >= @baseline_start_date_ and `date` <= @baseline_end_date_
        AND purchase_type = 'instore'
      GROUP BY product_id
    )
    , campaign_cost_tbl AS (
      -- guaranteed to have distinct product_id
        SELECT product_id
            , product_name
            , brand
            , category
            , IF(campaign_id IS NOT NULL, 1, 0) is_included_in_campaign
            , campaign_id
            , campaign_name
            , campaign_type
            , start_date
            , end_date
            , estimated_budget
            , approved_budget
            , real_spent_budget
            , discount
            , estimated_sales_increase
            , AVG(count_product) count_product # count of product included in this campaign as planned
            , SUM(cost_in_store) cost_in_store
            , SUM(cost_flyer) cost_flyer
            , SUM(cost_community) cost_community
            , SUM(cost_loyalty) cost_loyalty
            , SUM(cost_push) cost_push
            , SUM(cost_ad_total) cost_ad_total
      FROM (
          SELECT DISTINCT
                `date`
              , product_id
              , product_name
              , brand
              , category
              , IF(campaign_id IS NOT NULL, 1, 0) is_included_in_campaign
              , campaign_id
              , campaign_name
              , campaign_type
              , start_date
              , end_date
              , estimated_budget
              , approved_budget
              , real_spent_budget
              , discount
              , estimated_sales_increase
              , count_product # count of product included in this campaign as planned
                , cost_in_store
                , cost_flyer
                , cost_community
                , cost_loyalty
                , cost_push
                , cost_ad_total
          FROM `positive-karma-457703-i3.retail_dashboard.dws_campaign_cost_sale_prd_d_inc`
          WHERE `date` >= @campaign_start_date_# it is already been sumed up in the last dws table for 7 days. campaign_start_date_
            AND `date` <= @campaign_end_date_
        )
      GROUP BY product_id
            , product_name
            , brand
            , category
            , IF(campaign_id IS NOT NULL, 1, 0)
            , campaign_id
            , campaign_name
            , campaign_type
            , start_date
            , end_date
            , estimated_budget
            , approved_budget
            , real_spent_budget
            , discount
            , estimated_sales_increase
    )
    , halo_effect_tbl AS (
        SELECT product_id
            , IF(in_campaign_brand_tbl.brand IS NOT NULL, 1, 0) is_brand_halo_effect
            , IF(in_campaign_cat_tbl.category IS NOT NULL, 1, 0) is_category_halo_effect
        FROM `positive-karma-457703-i3.retail_dashboard.dim_product_full` AS prd_tbl
        LEFT JOIN (
          SELECT DISTINCT brand
          FROM campaign_cost_tbl
          WHERE campaign_id IS NOT NULL
        ) AS in_campaign_brand_tbl
      ON prd_tbl.brand = in_campaign_brand_tbl.brand
      LEFT JOIN (
        SELECT DISTINCT category
        FROM campaign_cost_tbl
        WHERE campaign_id IS NOT NULL
      ) AS in_campaign_cat_tbl
      ON prd_tbl.category = in_campaign_cat_tbl.category
    )
    , campaign_halo_cost_tbl AS (
        SELECT t1.brand
            , t1.category
            , halo_cost_in_store
            , halo_cost_flyer
            , halo_cost_community
            , halo_cost_loyalty
            , halo_cost_push
            , halo_cost_ad_total
        FROM
        (
          SELECT DISTINCT brand, category
          FROM campaign_cost_tbl
        ) t1
        LEFT JOIN (
        SELECT brand, category
            , AVG(cost_in_store) halo_cost_in_store
            , AVG(cost_flyer) halo_cost_flyer
            , AVG(cost_community) halo_cost_community
            , AVG(cost_loyalty) halo_cost_loyalty
            , AVG(cost_push) halo_cost_push
            , AVG(cost_ad_total) halo_cost_ad_total
        FROM campaign_cost_tbl
        WHERE campaign_id IS NOT NULL
        GROUP BY brand, category
        ) t0
        ON t0.brand = t1.brand
        AND t0.category = t1.category
    )

    SELECT campaign_cost_tbl.product_id
          , campaign_cost_tbl.product_name
          , campaign_cost_tbl.brand
          , is_brand_halo_effect
          , campaign_cost_tbl.category
          , is_category_halo_effect
          , campaign_cost_tbl.is_included_in_campaign
          , coalesce(campaign_cost_tbl.campaign_id, @campaign_id_) AS campaign_id
          , campaign_name
          , campaign_type
          , start_date
          , end_date
          , estimated_budget
          , approved_budget
          , real_spent_budget
          , discount
          , count_product
          , cost_in_store
          , cost_flyer
          , cost_community
          , cost_loyalty
          , cost_push
          , cost_ad_total
          , baseline_tbl.total_price_sum AS baseline_revenue
          , baseline_tbl.total_cost_sum AS baseline_cost
          , IF(campaign_id IS NOT NULL, post_campaign_tbl.total_discount_price_sum, post_campaign_tbl.total_price_sum) AS campaign_revenue # campaign use discounted sale price
          , post_campaign_tbl.total_cost_sum AS campaign_cost
          , estimated_sales_increase / 100 * baseline_tbl.total_price_sum AS estimated_revenue
          , IF(campaign_id IS NOT NULL, post_campaign_tbl.total_discount_price_sum, post_campaign_tbl.total_price_sum) - baseline_tbl.total_price_sum AS incremental_revenue
          , halo_cost_in_store
          , halo_cost_flyer
          , halo_cost_community
          , halo_cost_loyalty
          , halo_cost_push
          , halo_cost_ad_total
    FROM campaign_cost_tbl
    LEFT JOIN baseline_tbl
      ON campaign_cost_tbl.product_id = baseline_tbl.product_id
    LEFT JOIN post_campaign_tbl
      ON campaign_cost_tbl.product_id = post_campaign_tbl.product_id
    LEFT JOIN halo_effect_tbl
      ON campaign_cost_tbl.product_id = halo_effect_tbl.product_id
    LEFT JOIN campaign_halo_cost_tbl
      ON campaign_cost_tbl.brand = campaign_halo_cost_tbl.brand
      AND campaign_cost_tbl.category = campaign_halo_cost_tbl.category
    ORDER BY product_id ASC
"""


# Step 1: Check if table exists
def check_table(table_id_, full_table_id_, schema_, partition_column_):
  try:
      table = client.get_table(full_table_id_)
      print(f"    Table {table_id_} already exists.")
  except:
      print(f"    Creating partitioned table {table_id_}...")

      table = bigquery.Table(full_table_id_, schema=schema_)
      table.time_partitioning = bigquery.TimePartitioning(
          type_=bigquery.TimePartitioningType.DAY,
          field=partition_col
      )

      table = client.create_table(table)
      print(f"Created partitioned table {full_table_id_}")

# Step 2: Insert Query
def append_table(insert_query_, full_table_id_, query_parameters_):
  job_config = bigquery.QueryJobConfig(
      destination = full_table_id_,
      write_disposition=bigquery.WriteDisposition.WRITE_APPEND,
      use_legacy_sql=False,
      query_parameters=query_parameters_
  )
  query_job_ = client.query(insert_query_, job_config=job_config)  # Make an API request.

  print('   Result:', query_job_.result())

  print('   Errors: ', query_job_.errors)

  return(query_job_)

# Step 3: Check Insert Rows
def check_insert_rows(partition_col_, full_table_id_, param_):
  query = f"""
      SELECT {partition_col_}, count(1) row_cnt
      FROM {full_table_id_}
      WHERE {partition_col_} = @campaign_id_
      GROUP BY {partition_col_}
  """
  job_config = bigquery.QueryJobConfig(
      query_parameters=[
          # bigquery.ScalarQueryParameter("date_", "DATE", job_date_)
          bigquery.ScalarQueryParameter("campaign_id_", "INTEGER", param_)
      ]
  )
  query_job = client.query_and_wait(query, job_config=job_config)

  row_cnt_df = query_job.to_dataframe()
  return(row_cnt_df)

# Main excution
dataset_id = 'retail_dashboard'
target_table = 'ads_campaign_prd_in_store_full'
campaign_id_ = 1
window_len_ = 7
full_table_id = f"{client.project}.{dataset_id}.{target_table}"

parameter_query = f"""
    SELECT campaign_id # assupmtion: only one campaign happens during one period
       , start_date AS post_campaign_start_date
       , DATE_ADD(end_date, INTERVAL @window_len_ - 1 DAY) AS post_campaign_end_date # including today + the following 6 days = 1 week
       , DATE_SUB(start_date, INTERVAL campaign_length + @window_len_ DAY) AS baseline_start_date
       , DATE_SUB(start_date, INTERVAL 1 DAY) AS baseline_end_date
       , start_date AS campaign_start_date
       , end_date AS campaign_end_date
  FROM (
    SELECT DISTINCT campaign_id, start_date, end_date, DATE_DIFF(end_date, start_date, DAY) + 1 campaign_length
    FROM `positive-karma-457703-i3.retail_dashboard.dwd_campaign_d_inc`
    WHERE campaign_id = @campaign_id_
  ) AS t0
"""

print('- Execution campaign_id : ', campaign_id_)

print('- Read parameters :')
# Start the query, passing in the extra configuration.
query_job = client.query(parameter_query, job_config=bigquery.QueryJobConfig(query_parameters = [
        bigquery.ScalarQueryParameter("campaign_id_", "INTEGER", campaign_id_)
        , bigquery.ScalarQueryParameter("window_len_", "INTEGER", window_len_)
    ]))  # Make an API request.
para_df = query_job.result().to_dataframe()

post_campaign_start_date_ = para_df.loc[0]['post_campaign_start_date']
post_campaign_end_date_ = para_df.loc[0]['post_campaign_end_date']
baseline_start_date_ = para_df.loc[0]['baseline_start_date']
baseline_end_date_ = para_df.loc[0]['baseline_end_date']
campaign_start_date_ = para_df.loc[0]['campaign_start_date']
campaign_end_date_ = para_df.loc[0]['campaign_end_date']
print(para_df)


query_parameters = [
        bigquery.ScalarQueryParameter("campaign_id_", "INTEGER", campaign_id_),
        bigquery.ScalarQueryParameter("post_campaign_start_date_", "DATE", post_campaign_start_date_),
        bigquery.ScalarQueryParameter("post_campaign_end_date_", "DATE", post_campaign_end_date_),
        bigquery.ScalarQueryParameter("baseline_start_date_", "DATE", baseline_start_date_),
        bigquery.ScalarQueryParameter("baseline_end_date_", "DATE", baseline_end_date_),
        bigquery.ScalarQueryParameter("campaign_start_date_", "DATE", campaign_start_date_),
        bigquery.ScalarQueryParameter("campaign_end_date_", "DATE", campaign_end_date_)
    ]

# check_table(table_id_=target_table, full_table_id_=full_table_id, schema_ = table_schema, partition_column_ = partition_col)
query_job = append_table(insert_query_ = insert_query, full_table_id_ = full_table_id, query_parameters_ = query_parameters)
print('- Finish writing to campaign_id', campaign_id_)
row_cnt_df = check_insert_rows(partition_col_ = "campaign_id", full_table_id_ = full_table_id, param_ = 1)
print('- Check: Total ', str(row_cnt_df['row_cnt'].values[0]), 'rows.')

- Execution campaign_id :  1
- Read parameters :
   campaign_id post_campaign_start_date post_campaign_end_date  \
0            1               2024-01-13             2024-01-26   

  baseline_start_date baseline_end_date campaign_start_date campaign_end_date  
0          2023-12-29        2024-01-12          2024-01-13        2024-01-20  
[ScalarQueryParameter('campaign_id_', 'INTEGER', 1), ScalarQueryParameter('post_campaign_start_date_', 'DATE', datetime.date(2024, 1, 13)), ScalarQueryParameter('post_campaign_end_date_', 'DATE', datetime.date(2024, 1, 26)), ScalarQueryParameter('baseline_start_date_', 'DATE', datetime.date(2023, 12, 29)), ScalarQueryParameter('baseline_end_date_', 'DATE', datetime.date(2024, 1, 12)), ScalarQueryParameter('campaign_start_date_', 'DATE', datetime.date(2024, 1, 13)), ScalarQueryParameter('campaign_end_date_', 'DATE', datetime.date(2024, 1, 20))]
   Result: <google.cloud.bigquery.table.RowIterator object at 0x7e7c008f1ba0>
   Errors:  None
- Finish writ

## Iterate over campaign_id


In [25]:
# Main excution
dataset_id = 'retail_dashboard'
target_table = 'ads_campaign_prd_in_store_full'
campaign_id_start = 1
campaign_id_end = 20
window_len_ = 7
full_table_id = f"{client.project}.{dataset_id}.{target_table}"

parameter_query = f"""
    SELECT campaign_id # assupmtion: only one campaign happens during one period
       , start_date AS post_campaign_start_date
       , DATE_ADD(end_date, INTERVAL @window_len_ - 1 DAY) AS post_campaign_end_date # including today + the following 6 days = 1 week
       , DATE_SUB(start_date, INTERVAL campaign_length + @window_len_ DAY) AS baseline_start_date
       , DATE_SUB(start_date, INTERVAL 1 DAY) AS baseline_end_date
       , start_date AS campaign_start_date
       , end_date AS campaign_end_date
  FROM (
    SELECT DISTINCT campaign_id, start_date, end_date, DATE_DIFF(end_date, start_date, DAY) + 1 campaign_length
    FROM `positive-karma-457703-i3.retail_dashboard.dwd_campaign_d_inc`
    WHERE campaign_id = @campaign_id_
  ) AS t0
"""

total_row_cnt_df = pd.DataFrame()

# iterate over range of dates
while (campaign_id_start <= campaign_id_end):
  campaign_id_ = campaign_id_start

  print('- Execution campaign_id : ', campaign_id_)

  print('- Read parameters :')
  # Start the query, passing in the extra configuration.
  query_job = client.query(parameter_query, job_config=bigquery.QueryJobConfig(query_parameters = [
          bigquery.ScalarQueryParameter("campaign_id_", "INTEGER", campaign_id_)
          , bigquery.ScalarQueryParameter("window_len_", "INTEGER", window_len_)
      ]))  # Make an API request.
  para_df = query_job.result().to_dataframe()

  post_campaign_start_date_ = para_df.loc[0]['post_campaign_start_date']
  post_campaign_end_date_ = para_df.loc[0]['post_campaign_end_date']
  baseline_start_date_ = para_df.loc[0]['baseline_start_date']
  baseline_end_date_ = para_df.loc[0]['baseline_end_date']
  campaign_start_date_ = para_df.loc[0]['baseline_start_date']
  campaign_end_date_ = para_df.loc[0]['baseline_end_date']
  print(para_df)

  # check_table(table_id_=target_table, full_table_id_=full_table_id, schema_ = table_schema, partition_column_ = partition_col)

  query_parameters = [
          bigquery.ScalarQueryParameter("campaign_id_", "INTEGER", campaign_id_),
          bigquery.ScalarQueryParameter("post_campaign_start_date_", "DATE", post_campaign_start_date_),
          bigquery.ScalarQueryParameter("post_campaign_end_date_", "DATE", post_campaign_end_date_),
          bigquery.ScalarQueryParameter("baseline_start_date_", "DATE", baseline_start_date_),
          bigquery.ScalarQueryParameter("baseline_end_date_", "DATE", baseline_end_date_),
          bigquery.ScalarQueryParameter("campaign_start_date_", "DATE", campaign_start_date_),
          bigquery.ScalarQueryParameter("campaign_end_date_", "DATE", campaign_end_date_)
      ]

  query_job = append_table(insert_query_ = insert_query, full_table_id_ = full_table_id, query_parameters_ = query_parameters)
  print('- Finish writing to campaign_id', campaign_id_)
  row_cnt_df = check_insert_rows(partition_col_ = "campaign_id", full_table_id_ = full_table_id, param_ = campaign_id_)
  print('- Check: Total ', str(row_cnt_df['row_cnt'].values[0]), 'rows.')

  campaign_id_start += 1


- Execution campaign_id :  1
- Read parameters :
   campaign_id post_campaign_start_date post_campaign_end_date  \
0            1               2024-01-13             2024-01-26   

  baseline_start_date baseline_end_date campaign_start_date campaign_end_date  
0          2023-12-29        2024-01-12          2024-01-13        2024-01-20  
   Result: <google.cloud.bigquery.table.RowIterator object at 0x7e7c00d1c3a0>
   Errors:  None
- Finish writing to campaign_id 1
- Check: Total  2000 rows.
- Execution campaign_id :  2
- Read parameters :
   campaign_id post_campaign_start_date post_campaign_end_date  \
0            2               2024-02-05             2024-02-25   

  baseline_start_date baseline_end_date campaign_start_date campaign_end_date  
0          2024-01-14        2024-02-04          2024-02-05        2024-02-19  
   Result: <google.cloud.bigquery.table.RowIterator object at 0x7e7c01204700>
   Errors:  None
- Finish writing to campaign_id 2
- Check: Total  2000 rows.
- Ex

### Insert Summary

In [29]:
%%bigquery result
SELECT *
FROM `positive-karma-457703-i3.retail_dashboard.ads_campaign_prd_in_store_full`

Query is running:   0%|          |

Downloading:   0%|          |

In [30]:
result

,product_id,product_name,brand,is_brand_halo_effect,category,is_category_halo_effect,is_included_in_campaign,campaign_id,campaign_name,campaign_type,...,campaign_revenue,campaign_cost,estimated_revenue,incremental_revenue,halo_cost_in_store,halo_cost_flyer,halo_cost_community,halo_cost_loyalty,halo_cost_push,halo_cost_ad_total
0,1,Cost Heart,Local Farms,1,Frozen Foods,1,0,2,None,None,...,7214.13,4005.54,NaN,-4028.67,222.478566,0.0,0.000000,246.246966,324.217188,NaN
1,2,Not Bit,CPG3,1,Dairy,1,1,1,Sales Promotion Campaign Response,Sales Promotion Campaign,...,NaN,236.64,18.88215,NaN,221.655650,0.0,0.000000,387.337442,402.880796,NaN
2,2,Not Bit,CPG3,1,Dairy,1,0,2,None,None,...,425.72,236.64,NaN,-344.98,221.655650,0.0,0.000000,387.337442,402.880796,NaN
3,3,Easy Poor,Western Family,1,Snacks,1,0,2,None,None,...,7190.55,3992.43,NaN,578.55,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Join Think,Western Family,1,Meat,1,0,2,None,None,...,2605.65,1446.81,NaN,-239.60,278.771608,0.0,0.000000,241.937305,218.352745,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42343,1996,Room Modern,Local Farms,1,Deli,1,0,16,None,None,...,2160.94,1199.95,NaN,-1699.38,81.181616,0.0,136.940404,139.828642,142.387714,41.5
42344,1997,Avoid School,CPG2,1,Frozen Foods,1,0,16,None,None,...,12008.04,6667.32,NaN,1273.58,111.831554,0.0,88.270867,107.893670,114.761512,205.2
42345,1998,Use Sort,Local Farms,1,Meat,1,0,16,None,None,...,10145.24,5632.55,NaN,915.36,118.506239,0.0,61.488919,130.634185,96.359819,294.0
42346,1999,Say Rest,CPG2,1,Bakery,1,0,16,None,None,...,7475.60,4150.30,NaN,1359.20,178.219643,0.0,90.654943,100.155643,115.733452,100.5
